In [1]:
!pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 5.5 MB/s eta 0:00:00


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 23.5 MB/s eta 0:00:00


**IMPORTS**

In [ ]:
from pathlib import Path
import numpy as np
import math
import os
import pandas as pd
import json
from enum import Enum
import time
from typing import List, Generator, Dict, Any, Tuple, Union, Set
from types import SimpleNamespace
from dataclasses import dataclass
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import sys
import itertools

from IPython.display import display, Image
from shapely.geometry import box
from shapely.ops import unary_union

In [6]:
import humanize
import psutil
import GPUtil
import supervision as sv
import torch
from ultralytics import YOLO
from ultralytics.engine.results import Results
from supervision.detection.core import Detections
from supervision.tracker.byte_tracker.core import ByteTrack

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
class Labels(Enum):
    gallina_bianca = 0
    gallina_rossa = 1

In [ ]:
# Define the file name and URL
file_name = "ExternDisk0_ch4_20240207130000_20240207140000.mp4"
url = "https://www.dropbox.com/scl/fi/rwz8irhljs0dm08yy2zku/ExternDisk0_ch4_20240207130000_20240207140000.mp4?rlkey=fi02g6j9hlet481hw9kk4inru&st=08kvj8ln&dl=0"

# Download the file using wget
os.system(f"wget -O {file_name} '{url}'")

print(f"File '{file_name}' scaricato nella directory corrente.")

In [ ]:
best_weights = os.path.join(
    'runs', 'detect', 'train_2_yolo11n', 'weights', 'best.pt')

base_folder = os.path.join('raw_video_source')

In [ ]:
####
input_video_path = os.path.join(base_folder, "ExternDisk0_ch4_20240207130000_20240207140000.mp4")

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.empty_cache()

weight_file = os.path.join(base_folder, "runs", "detect", "train_2_yolo11n", "weights", "best.pt")
model = YOLO(weight_file)
model.to(device)

**OBJECT DETECTION SU IMMAGINE**

In [ ]:
image_path = os.path.join('splitted_cut_version', 'train', 'images', 'ExternDisk0_ch4_animal_03_frame_03402.jpg')
image = cv2.imread(image_path)
results = model(image)[0]
detections = sv.Detections.from_ultralytics(results)

bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

labels = [
    model.model.names[class_id]
    for class_id
    in detections.class_id
]

annotated_image = bounding_box_annotator.annotate(
    scene=image, detections=detections)
annotated_image = label_annotator.annotate(
    scene=annotated_image, detections=detections, labels=labels)

sv.plot_image(annotated_image)

**OBJECT DETECTION SU VIDEO STREAM**

In [ ]:
box_annotator = sv.BoundingBoxAnnotator()

def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    return box_annotator.annotate(frame.copy(), detections=detections)

output_video_path = os.path.join(
    base_folder, "ExternDisk0_ch4_20240207130000_20240207140000_annotated_1.mp4")

sv.process_video(
    source_path=input_video_path,
    target_path=output_video_path,
    callback=callback
)
del box_annotator

**OBJECT TRACKING SU VIDEO STREAM**

In [ ]:
tracker = sv.ByteTrack()
box_annotator = sv.BoundingBoxAnnotator()

def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)
    return box_annotator.annotate(frame.copy(), detections=detections)


output_video_path = os.path.join(
    base_folder, "ExternDisk0_ch4_20240207130000_20240207140000_annotated_2.mp4")

sv.process_video(
    source_path=input_video_path,
    target_path=output_video_path,
    callback=callback
)

del tracker
del box_annotator

**OBJECT TRACKING SU VIDEO STREAM CON LABEL SUI BOUNDING BOX**

In [ ]:
tracker = sv.ByteTrack()
box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()


def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    labels = [
        f"#{tracker_id} {results.names[class_id]}"
        for class_id, tracker_id
        in zip(detections.class_id, detections.tracker_id)
    ]

    annotated_frame = box_annotator.annotate(
        frame.copy(), detections=detections)
    return label_annotator.annotate(
        annotated_frame, detections=detections, labels=labels)


output_video_path = os.path.join(
    base_folder, "ExternDisk0_ch4_20240207130000_20240207140000_annotated_3.mp4")

sv.process_video(
    source_path=input_video_path,
    target_path=output_video_path,
    callback=callback
)

del label_annotator
del tracker
del box_annotator